***Import packages***

In [1]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')

In [2]:
from rackuten_project_lib import *

In [5]:
class Config:
    
    images_Dir =            'D:/DataScienTest_WorkSpace/WS/data/images/all/image_train/'   
    model_dir  =            'D:/DataScienTest_WorkSpace/WS/Travail_Final/Trained_models/'
    df_Dir =                'D:/DataScienTest_WorkSpace/WS/Travail_Final/Datasets/'
    output_dir =            'D:/DataScienTest_WorkSpace/WS/Travail_Final/Outputs/InceptionV3_Simple_DNN_Conv1D/'
   
    
    #xception_fname =        'Model_Xception_All_train_data_Shape_299x299_100Epochs_Step6.hdf5'
    inception_fname =        'Model_InceptionV3_All_train_data_Shape_299x299_100Epochs_Step2.hdf5'
    simpleDNN_fname  =       'Model_SimpleDNN_All_train_data_100Epochs.hdf5'    
    conv1D_fname  =          'Model_Conv1D_All_train_data_50Epochs.hdf5'
    
    data_train =            'df_train_final.pkl'
    
    inception_im_shape = (299, 299)

    inception_SC = 0.64
    conv1D_SC = 0.80
    simplDNN_SC = 0.81
 

In [6]:
config = Config()

# Load cleaned DF

In [7]:
data_source  = pd.read_pickle(config.df_Dir + config.data_train )

In [8]:
display(data_source.head(4))

,text,prdtypecode_org,prdtypecode,labels,image_name
0,olivia personalisiertes notizbuch seiten punkt...,10,0,adult books,image_1263597046_product_3804725264.jpg
1,journal arts art marche salon art asiatique pa...,2280,18,magazines,image_1008141237_product_436067568.jpg
2,grand stylet ergonomique bleu gamepad nintendo...,50,2,video games accessories,image_938777978_product_201115110.jpg
3,peluche donald europe disneyland marionnette d...,1280,7,toys for children,image_457047496_product_50418756.jpg


# Load pre-trained models

In [9]:
def load_trained_model(model_fname):
    
    model = load_model(config.model_dir + model_fname ,  compile = True )
    
    return model 

In [10]:
# Load Pre_trained CNN Model - InceptionV3 
inceptionNet = load_trained_model(config.inception_fname)
#inceptionNet.summary()

In [11]:
# Load Pre_trainde Model Simple DNN 
simpleDnnNet = load_trained_model(config.simpleDNN_fname)
#simpleDnnNet.summary()

In [12]:
# Load Pre_trainde Model Conv1D
con1DNet = load_trained_model(config.conv1D_fname)
#con1DNet.summary()

# Create Train and Test sets 

In [13]:
X_train, X_test = train_test_split(data_source, test_size=0.2, random_state = 1234)

In [14]:
X_test['fullpath'] = config.images_Dir + X_test['image_name']
y_test_true = X_test['prdtypecode']

In [15]:
X_test.head(1)

,text,prdtypecode_org,prdtypecode,labels,image_name,fullpath
66047,stabilo lot stylos feutre film ohpen universal...,2522,21,stationery,image_984686659_product_289483356.jpg,D:/DataScienTest_WorkSpace/WS/data/images/all/...


In [16]:
def tokenize_text(X_train_text, X_test_text):
  
    #maxlen = 500
    maxlen = 400
    num_words = 20000
    
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = num_words)
    tokenizer.fit_on_texts(X_train_text)
    
    X_train = tokenizer.texts_to_sequences(X_train_text)
    X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train,
                                                            maxlen = maxlen,
                                                            padding='post')
    X_test = tokenizer.texts_to_sequences(X_test_text)
    
    X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test,
                                                           maxlen = maxlen,
                                                           padding='post')
    print("X_train text created. Size : ", X_train.shape)
    print("X_test text created. Size : ", X_test.shape)    
    
    return X_train, X_test 

In [17]:
def tokenize_text_test_only(X_test_text):
  
    #maxlen = 500
    maxlen = 400
    num_words = 20000
    
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = num_words)
    tokenizer.fit_on_texts(X_test_text)
    
    X_test = tokenizer.texts_to_sequences(X_train_text)
    X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test,
                                                            maxlen = maxlen,
                                                            padding='post')

    print("X_test text created. Size : ", X_test.shape)    
    
    return X_train, X_test 

In [18]:
def predict_images(model,im_shape, fullpath):
    
    #fct_name = '[INFO]-[predict_images_proba]'

    y_pred_proba_list = []
    
    for filepath in tqdm(fullpath):
        im = cv2.resize(cv2.imread(filepath), im_shape)
        im = im/255
        im = np.expand_dims(im, axis = 0)
        
        out = model.predict(im)
        
        y_pred_proba_list.append(out)
        
      
    y_pred_proba = np.array(y_pred_proba_list)
    y_pred_proba = y_pred_proba.reshape(y_pred_proba.shape[0],27)
    
    permutation = [0, 1, 12, 20, 21, 22, 23, 24, 25, 26, 2, 3,
                   4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16
                   ,17, 18, 19]
    # swap columns matchs cnn label with real class number
    y_pred_proba = y_pred_proba[:, permutation] 
    y_pred_class = np.argmax(y_pred_proba,axis = 1).astype(int)  # pred with real class labels
    
    #display(fct_name + 'Probabilities real class num shape:' ,y_pred_proba.shape)
    #display(fct_name + 'Predicted real class labels(0 to 26) shape:',y_pred_class.shape)
    
    return y_pred_proba,  y_pred_class

In [21]:
def predict_images_data_gen(model, test_df, input_shape, image_Dir):    
 
        test_df.prdtypecode =   test_df.prdtypecode.astype(str)


        
        datagen = ImageDataGenerator(
                                    #tf.keras.applications.mobilenet.preprocess_input,
                                    rescale=1./255
                                    )
        
           
        test_img_gen = datagen.flow_from_dataframe(dataframe = test_df, 
                                                    directory = image_Dir,
                                                    x_col='image_name', 
                                                    y_col='prdtypecode',                                                                                 
                                                    class_mode='sparse',
                                                    target_size=(input_shape[0], input_shape[1]),
                                                    batch_size= 64, 
                                                    shuffle = False                                                   
                                                   )
        
        y_pred_proba = get_true_class(model, test_img_gen)
        return y_pred_proba

def get_true_class(model, data_gen):
    
    data_gen.reset()
    
    #test_score_cnn = model.evaluate(data_gen)
    
    pred = model.predict(data_gen, verbose = 1)
  
    class_dict =  data_gen.class_indices  # a dictionary where key is the class name (  assigned by Image Data Generator)
                                           #and value is the corresponding label of real class of dataset of images
        
   # List of permutation corresponding to real class number
   
    permutation = [data_gen.class_indices[str(i)] for i in range(len(data_gen.class_indices))]
    
    #print(permutation)   
    
    y_pred_proba = pred[:, permutation] # swap columns
    pred_class_cnn = np.argmax(y_pred_proba, axis = 1).astype(int)
    
    return y_pred_proba, pred_class_cnn

# Predict with Images - InceptionV3

***Predict Images One by One***

In [19]:
y_pred_proba_cnn_1, pred_class_cnn_1 = predict_images(inceptionNet,
                                                  config.inception_im_shape,
                                                  X_test['fullpath'])

100%|██████████| 16984/16984 [20:24<00:00, 13.87it/s]


***Predict with data Generator***

In [22]:
y_pred_proba_cnn_2, pred_class_cnn_2 = predict_images_data_gen(inceptionNet,
                                                           X_test,                                                          
                                                           config.inception_im_shape,
                                                           config.images_Dir)

Found 16984 validated image filenames belonging to 27 classes.
266/266 [==============================] - 77s 281ms/step


***Final CNN prediction***

In [27]:
#y_pred_proba_cnn = y_pred_proba_cnn_1
#pred_class_cnn =  pred_class_cnn_1

y_pred_proba_cnn = y_pred_proba_cnn_2
pred_class_cnn =  pred_class_cnn_2

In [28]:
# With real class num
print(metrics.classification_report(y_test_true , pred_class_cnn))

              precision    recall  f1-score   support

           0       0.59      0.61      0.60       643
           1       0.69      0.60      0.65       503
           2       0.52      0.34      0.41       319
           3       0.70      0.82      0.76       179
           4       0.58      0.74      0.65       545
           5       0.94      0.89      0.92       793
           6       0.60      0.31      0.41       144
           7       0.52      0.41      0.46      1002
           8       0.36      0.28      0.32       408
           9       0.65      0.81      0.72       993
          10       0.80      0.51      0.62       162
          11       0.62      0.37      0.47       514
          12       0.56      0.48      0.51       664
          13       0.61      0.54      0.57       993
          14       0.66      0.88      0.76       832
          15       0.38      0.73      0.50       159
          16       0.46      0.58      0.51       970
          17       0.43    

# 

# Predict withText - Simple DNN &  Embedding + Conv1D

In [29]:
X_train_text, X_test_text = tokenize_text(X_train['text'], X_test['text'])

X_train text created. Size :  (67932, 400)
X_test text created. Size :  (16984, 400)


***Simple DNN***

In [30]:
y_pred_proba_rnn_1 = simpleDnnNet.predict(X_test_text)
y_pred_rnn_1_class = np.argmax(y_pred_proba_rnn_1,axis = 1).astype(int)

In [31]:
# Report Text predictions
print(metrics.classification_report(y_test_true, y_pred_rnn_1_class))

              precision    recall  f1-score   support

           0       0.40      0.62      0.49       643
           1       0.75      0.62      0.68       503
           2       0.77      0.86      0.81       319
           3       0.97      0.82      0.89       179
           4       0.77      0.80      0.79       545
           5       0.87      0.90      0.88       793
           6       0.75      0.47      0.58       144
           7       0.72      0.73      0.73      1002
           8       0.68      0.54      0.60       408
           9       0.94      0.94      0.94       993
          10       0.97      0.88      0.92       162
          11       0.84      0.77      0.80       514
          12       0.79      0.78      0.79       664
          13       0.80      0.86      0.83       993
          14       0.86      0.95      0.90       832
          15       0.85      0.80      0.82       159
          16       0.83      0.78      0.80       970
          17       0.93    

***Embedding + Conv1D***

In [32]:
#CONV1D
y_pred_proba_rnn_2 = con1DNet.predict(X_test_text)
y_pred_rnn_2_class = np.argmax(y_pred_proba_rnn_2,axis = 1).astype(int)

In [33]:
# Report Text predictions
print(metrics.classification_report(y_test_true, y_pred_rnn_2_class))

              precision    recall  f1-score   support

           0       0.40      0.56      0.47       643
           1       0.71      0.65      0.68       503
           2       0.83      0.76      0.79       319
           3       0.88      0.84      0.86       179
           4       0.73      0.81      0.77       545
           5       0.84      0.89      0.86       793
           6       0.89      0.50      0.64       144
           7       0.70      0.74      0.72      1002
           8       0.63      0.51      0.56       408
           9       0.95      0.93      0.94       993
          10       0.99      0.89      0.94       162
          11       0.80      0.76      0.78       514
          12       0.82      0.75      0.79       664
          13       0.85      0.81      0.83       993
          14       0.90      0.96      0.92       832
          15       0.84      0.81      0.82       159
          16       0.79      0.82      0.81       970
          17       0.84    

# 

#  Models Voting

Max Voting </br>
Weighted Average </br>

***Max Voting***

In [34]:
final_pred_max_proba =np.maximum(y_pred_proba_cnn, y_pred_proba_rnn_1, y_pred_proba_rnn_2)

In [35]:
final_pred_max = np.argmax(final_pred_max_proba,axis = 1).astype(int)

In [36]:
print(metrics.classification_report(y_test_true, final_pred_max))

              precision    recall  f1-score   support

           0       0.69      0.63      0.66       643
           1       0.86      0.75      0.80       503
           2       0.80      0.78      0.79       319
           3       0.90      0.89      0.90       179
           4       0.74      0.86      0.79       545
           5       0.94      0.98      0.96       793
           6       0.79      0.47      0.59       144
           7       0.76      0.70      0.73      1002
           8       0.66      0.50      0.57       408
           9       0.90      0.96      0.93       993
          10       0.96      0.88      0.92       162
          11       0.86      0.76      0.80       514
          12       0.83      0.75      0.79       664
          13       0.81      0.85      0.83       993
          14       0.83      0.95      0.88       832
          15       0.66      0.87      0.75       159
          16       0.80      0.77      0.79       970
          17       0.82    

***Weighted Average***

In [37]:
final_pred_weighted_proba = (y_pred_proba_cnn * (config.inception_SC) +    
                            y_pred_proba_rnn_1 * (config.simplDNN_SC) +  
                            y_pred_proba_rnn_2 * (config.conv1D_SC))/(config.inception_SC + config.simplDNN_SC + config.conv1D_SC)


In [38]:
final_pred_weighted = np.argmax(final_pred_weighted_proba,axis = 1).astype(int)

In [39]:
print(metrics.classification_report(y_test_true, final_pred_weighted))

              precision    recall  f1-score   support

           0       0.67      0.65      0.66       643
           1       0.85      0.75      0.80       503
           2       0.82      0.82      0.82       319
           3       0.93      0.89      0.91       179
           4       0.75      0.86      0.80       545
           5       0.94      0.98      0.96       793
           6       0.84      0.47      0.60       144
           7       0.74      0.72      0.73      1002
           8       0.69      0.52      0.59       408
           9       0.91      0.96      0.93       993
          10       0.97      0.88      0.92       162
          11       0.86      0.76      0.81       514
          12       0.84      0.76      0.80       664
          13       0.82      0.86      0.84       993
          14       0.84      0.95      0.89       832
          15       0.71      0.89      0.79       159
          16       0.80      0.79      0.80       970
          17       0.86    

***Save results***

In [40]:
def get_org_class_code(y_pred):

    df_code = pd.DataFrame(y_pred)

    #display('[INFO]-[get_class_code] BEFORE changing Class numbers :',df_code)
    df_code.replace({0: 
                        {0:10, 1:40, 2:50, 3:60, 4:1140,
                        5:1160, 6:1180, 7:1280, 8:1281,
                        9:1300, 10:1301, 11:1302, 12:1320,
                        13:1560, 14:1920, 15:1940, 16:2060,
                        17:2220, 18:2280, 19:2403, 20:2462,
                        21:2522, 22:2582, 23:2583, 24:2585,
                        25:2705, 26:2905,
                        }}, inplace = True)

    #display('[INFO]-[get_class_code] AFTER changing Class numbers :',df_code)
    
    return df_code

In [41]:
y_test_true_code = X_test['prdtypecode_org']

In [43]:
labels = ['adult books',
        'imported video games',
        'video games accessories',
        'games and consoles',
        'figurines and Toy Pop',
        'playing cards',
        'figurines, masks and role playing games',
        'toys for children',
        'board games',
        'remote controlled models',
        'accessories children',
        'toys, outdoor playing, clothes',
        'early childhood',
        'interior furniture and bedding',
        'interior accessories',
        'Food',
        'decoration interior',
        'supplies for domestic animals',
        'magazines',
        'children books and magazines',
        'games',
        'stationery',
        'furniture kitchen and garden',
        'piscine spa',
        'gardening and DIY',
        'books',
        'online distribution of video games',
        '',
        '',
        '',] 


In [44]:
# Single CNN pred with org class code
df_pred_cnn_org = get_org_class_code(pred_class_cnn)

report = metrics.classification_report(y_test_true, pred_class_cnn, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df = report_df.round(2)
report_df['support'] = report_df['support'].astype(int)

report_cnn_org_cls = metrics.classification_report(y_test_true_code, df_pred_cnn_org, output_dict=True)
df_org_cls = pd.DataFrame(report_cnn_org_cls).transpose()
df_org_cls = df_org_cls.round(2)
#df_org_cls['support'] = df_org_cls['support'].astype(int)


df_report = report_df[['precision', 'recall', 'f1-score', 'support']]
#df_report['Class_number'] = report_df.index
df_report['Class_Code'] = df_org_cls.index
df_report = df_report[['Class_Code','precision', 'recall','f1-score', 'support']]
df_report = df_report.rename(columns={"precision": "precision_Inception",
                                      "recall": "recall_Inception",
                                      "f1-score": "f1-score_Inception"})
df_report['Labels'] = labels

# Single Simple DNN , Conv1D pred with org class code

report_rnn1 = metrics.classification_report(y_test_true, y_pred_rnn_1_class, output_dict=True)
df_rnn1 = pd.DataFrame(report_rnn1).transpose()
df_rnn1 = df_rnn1.round(2)


report_rnn2 = metrics.classification_report(y_test_true, y_pred_rnn_2_class, output_dict=True)
df_rnn2 = pd.DataFrame(report_rnn2).transpose()
df_rnn2 = df_rnn2.round(2)


#df_pred_W = get_org_class_code(final_pred_weighted)

report_vote_max = metrics.classification_report(y_test_true, final_pred_max, output_dict=True)
df_vote_max = pd.DataFrame(report_vote_max).transpose()
df_vote_max = df_vote_max.round(2)

report_vote_w = metrics.classification_report(y_test_true, final_pred_weighted, output_dict=True)
df_vote_w = pd.DataFrame(report_vote_w).transpose()
df_vote_w = df_vote_w.round(2)

#precision    recall  f1-score   support
df_report['f1-score_Simple_DNN'] = df_rnn1['f1-score']
df_report['precision_Simple_DNN'] = df_rnn1['precision']
df_report['recall_Simple_DNN'] = df_rnn1['recall']


df_report['f1-score_Conv1D'] = df_rnn2['f1-score']
df_report['precision_Conv1D'] = df_rnn2['precision']
df_report['recall_Conv1D'] = df_rnn2['recall']


df_report['f1-score_max_voting'] = df_vote_max['f1-score']
df_report['f1-score_weighted_voting'] = df_vote_w['f1-score']

col_order = ['Class_Code', 'Labels','precision_Inception', 'recall_Inception','f1-score_Inception',
             'f1-score_Simple_DNN', 'precision_Simple_DNN','recall_Simple_DNN',
             'precision_Conv1D', 'recall_Conv1D', 'f1-score_Conv1D', 
             'f1-score_max_voting', 'f1-score_weighted_voting', 'support']


df_report = df_report[col_order]
df_report.info()


<class 'pandas.core.frame.DataFrame'>
Index: 30 entries, 0 to weighted avg
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Class_Code                30 non-null     object 
 1   Labels                    30 non-null     object 
 2   precision_Inception       30 non-null     float64
 3   recall_Inception          30 non-null     float64
 4   f1-score_Inception        30 non-null     float64
 5   f1-score_Simple_DNN       30 non-null     float64
 6   precision_Simple_DNN      30 non-null     float64
 7   recall_Simple_DNN         30 non-null     float64
 8   precision_Conv1D          30 non-null     float64
 9   recall_Conv1D             30 non-null     float64
 10  f1-score_Conv1D           30 non-null     float64
 11  f1-score_max_voting       30 non-null     float64
 12  f1-score_weighted_voting  30 non-null     float64
 13  support                   30 non-null     int32  
dtypes: floa

In [45]:
date = '20022022'

df_pred_cnn = pd.DataFrame(pred_class_cnn)
df_pred_rnn1 = pd.DataFrame(y_pred_rnn_1_class)
df_pred_rnn2 = pd.DataFrame(y_pred_rnn_2_class)


df_final_pred_max = pd.DataFrame(final_pred_max)
df_final_pred_weighted = pd.DataFrame(final_pred_weighted)

# Single models

 #Images
df_pred_cnn.to_csv(f'{config.output_dir}/inception_Predictions_'+ date + '.csv')


 #Text
df_pred_rnn1.to_csv(f'{config.output_dir}/Simple_DNN_Predictions_'+ date + '.csv')
df_pred_rnn2.to_csv(f'{config.output_dir}/Conv1D_Predictions_'+ date + '.csv')


#Voting
 #Max Voting
df_final_pred_max.to_csv(f'{config.output_dir}/Max_voting_Inception_Simple_DNN_Con1D_'+ date + '.csv')

 #Weighted Average
df_final_pred_weighted.to_csv(f'{config.output_dir}/Weighted_voting_inception_Simple_DNN_Con1D_'+ date + '.csv')


#Summury report
df_report.to_csv(f'{config.output_dir}/Summury_Voting_Inception_Simple_DNN_Con1D_'+ date + '.csv')
